# Simulation #2
Having written the functions required to implement estimation of $L$, we now experiment with a few different model parameters. First, however, it will be helpful to wrap all of the helper functions into a single ```estimate_L``` function.

In [49]:
include("functions.jl");
include("nnlsq.jl");

In [32]:
state_space = Dict("min"=>0, "max"=>2π, "grid_resolution"=>1000);

In [50]:
L = estimate_L(state_space, 10000, S, φ, 1, 30, 100; dist="normal")

Academic license - for non-commercial use only - expires 2021-08-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 900 rows, 10900 columns and 8875004 nonzeros
Model fingerprint: 0x181e5049
Model has 900 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 2.42s
Presolved: 900 rows, 10900 columns, 8875004 nonzeros
Presolved model has 900 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.046e+05
 Factor NZ  : 4.054e+05 (roughly 8 MBytes of memory)
 Factor Ops : 2.434e+08 (roughly 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal      

900×900 Matrix{Float64}:
 0.000360148  0.00081329   0.000650557  …  4.24323e-9   1.67112e-9
 0.000473509  0.00107802   0.000873179     9.72292e-9   3.83887e-9
 0.000572066  0.00131742   0.00108567      2.38734e-8   9.52696e-9
 0.000637492  0.00149135   0.00125749      6.01637e-8   2.43225e-8
 0.000658448  0.00157274   0.00136513      1.50974e-7   6.16817e-8
 0.000634035  0.00155464   0.00139692   …  3.73991e-7   1.53646e-7
 0.000572802  0.00144888   0.00135296      9.1815e-7    3.7709e-7
 0.000488451  0.00127863   0.00124215      2.24198e-6   9.16097e-7
 0.000395042  0.00107062   0.00107956      5.4166e-6    2.19676e-6
 0.000303902  0.000850076  0.000884985     1.27631e-5   5.1383e-6
 0.000222691  0.000638695  0.000681227  …  2.8813e-5    1.1534e-5
 0.000155681  0.000453253  0.000490584     6.13575e-5   2.4472e-5
 0.000104365  0.000304206  0.000330226     0.000121919  4.85345e-5
 ⋮                                      ⋱               
 1.96686e-5   5.19301e-5   4.92287e-5      0.000254